In [1]:
!pip install -q tflite-model-maker
!pip install -q "requests_cache==0.5.2"
!pip install -q pygbif
!pip install -q qwikidata

In [2]:
import pygbif
from qwikidata.sparql import (get_subclasses_of_item,
                              return_sparql_query_results)
from qwikidata.entity import WikidataItem, WikidataLexeme, WikidataProperty
import qwikidata
import pandas as pd
from tqdm import tqdm
import os
from IPython.display import JSON
import logging
import time
import glob
logging.basicConfig(level=logging.DEBUG)
tqdm.pandas()

In [3]:
!ln -s /content/drive/My\ Drive/My\ work/Natur\ i\ Norge/Model /content/mydrive

# Get data

In [4]:
# species_list = pd.read_excel('raw/arter_AR8.xlsx', sheet_name='Artsliste')
species_list = pd.read_csv('partial/02-with-all-gbif-ids.csv')

In [5]:
species_list

,Unnamed: 0,Latinsk_navn,Norsk_navn,Total,Input,Unnamed: 4,taxonID,scientificNameID,scientificName,scientificNameAuthorship,...,family,genus,subgenus,specificEpithet,infraspecificEpithet,higherClassification,vernacularName (nb),vernacularName (nn),dynamicProperty,gbif_taxon_id
0,0,Abietinella abietina,granmose,17,Abietinella abietina,NaN,64749,104789,Abietinella abietina,(Hedw.) M.Fleisch.,...,Thuidiaceae,Abietinella,NaN,NaN,NaN,Plantae Bryophyta Bryopsida Hypnales Thuidiace...,granmose,granmose,NaN,5283402
1,1,Acarospora sinopica,rustsprekklav,1,Acarospora sinopica,NaN,41236,65263,Acarospora sinopica,(Wahlenb.) Körber,...,Acarosporaceae,Acarospora,NaN,NaN,NaN,Fungi Ascomycota Pezizomycotina Lecanoromycete...,rustsprekklav,NaN,NaN,2600524
2,2,Acaulon muticum,vortesvøpmose,1,Acaulon muticum,NaN,64842,104882,Acaulon muticum,(Hedw.) Müll.Hal.,...,Pottiaceae,Acaulon,NaN,NaN,NaN,Plantae Bryophyta Bryopsida Pottiales Pottiace...,vortesvøpmose,vortesveipmose,NaN,5280933
3,3,Acer platanoides,spisslønn,1,Acer platanoides,NaN,63519,103559,Acer platanoides,L.,...,Sapindaceae,Acer,NaN,NaN,NaN,Plantae Magnoliophyta Eudicots Sapindales Sapi...,spisslønn,spisslønn,NaN,3189846
4,4,Achillea millefolium,ryllik,20,Achillea millefolium,NaN,145524,100392,Achillea millefolium,L.,...,Asteraceae,Achillea,NaN,NaN,NaN,Plantae Magnoliophyta Eudicots Asterales Aster...,ryllik,ryllik,NaN,3120060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205,1205,Xanthoparmelia conspersa,stiftsteinlav,1,Xanthoparmelia conspersa,NaN,104800,73043,Xanthoparmelia conspersa,(Ach.) Hale,...,Parmeliaceae,Xanthoparmelia,NaN,NaN,NaN,Fungi Ascomycota Pezizomycotina Lecanoromycete...,stiftsteinlav,NaN,NaN,2604379
1206,1206,Xanthoparmelia pulla,skålskjærgårdslav,1,Xanthoparmelia pulla,NaN,104802,145208,Xanthoparmelia pulla,(Ach.) O.Blanco et al.,...,Parmeliaceae,Xanthoparmelia,NaN,NaN,NaN,Fungi Ascomycota Pezizomycotina Lecanoromycete...,skålskjærgårdslav,NaN,NaN,2604700
1207,1207,Xanthoparmelia stenophylla,gul steinlav,3,Xanthoparmelia stenophylla,NaN,49232,73259,Xanthoparmelia stenophylla,(Ach.) Ahti & D. Hawksw.,...,Parmeliaceae,Xanthoparmelia,NaN,NaN,NaN,Fungi Ascomycota Pezizomycotina Lecanoromycete...,gul steinlav,NaN,NaN,3407931
1208,1208,Xanthoria aureola,kystmessinglav,2,Xanthoria aureola,NaN,57757,81784,Xanthoria aureola,(Ach.) Erichsen,...,Teloschistaceae,Xanthoria,NaN,NaN,NaN,Fungi Ascomycota Pezizomycotina Lecanoromycete...,kystmessinglav,NaN,NaN,2609974


Get TaxonKey from GBIF using WikiData

In [5]:
def get_gbif_taxon_key(nbic_taxon_id):
  # logging.info(nbic_taxon_id)
  sparql_query = f"""
  SELECT ?item ?label
        WHERE {{ ?item wdt:P8707 '{nbic_taxon_id}'.
        ?item wdt:P846 ?label.
        SERVICE wikibase:label {{ bd:serviceParam wikibase:language '[AUTO_LANGUAGE],en'. }}
  }}
  """
  resp = None
  while resp == None:
    try:
      resp = return_sparql_query_results(sparql_query)
    except:
      pass
  time.sleep(0.2)
  # logging.info(resp)
  # return resp
  res = resp['results']['bindings']
  if len(res) == 1:
    return res[0]['label']['value']
  if len(res) == 0:
    logging.warning(f'No matches for nbic_taxon_id: {nbic_taxon_id}')
    return None
  else:
    logging.error(f'Multiple matches for nbic_taxon_id: {nbic_taxon_id}')
    return None
  
res = get_gbif_taxon_key(103217) # 104789
res

'3001956'

In [ ]:
species_list['gbif_taxon_id'] = species_list.scientificNameID.progress_apply(get_gbif_taxon_key)
species_list.to_csv('/content/mydrive/Model/partial/01-with-most-gbif-ids.csv')

fill out the missing keys using GBIF search

In [27]:
def search_gbif_taxon_key(name, gbif_taxon_key):
  if gbif_taxon_key != None:
    return gbif_taxon_key
  resp = pygbif.name_lookup(q=name)
  if resp['count'] == 0:
    logging.error(f'Name not found: {name}')
    return None
  results = resp['results']
  # return results
  for result in results:
    if 'taxonomicStatus' in result.keys():
      if result['taxonomicStatus'] == 'ACCEPTED':
        return result['key']
    if 'acceptedKey' in result.keys():
      return result['acceptedKey']
  
  logging.error(f'Unable to find match: {name}')

  return None;

search_gbif_taxon_key('Ranunculus auricomus agg.', None)

116803026

In [28]:
species_list['gbif_taxon_id'] = species_list.progress_apply(lambda x: search_gbif_taxon_key(x['scientificName'], x['gbif_taxon_id']), axis=1)
species_list.to_csv('/content/mydrive/partial/02-with-all-gbif-ids.csv')



100%|██████████| 1210/1210 [00:00<00:00, 1547.83it/s]


In [8]:
def initiate_gbif_download(taxon_id):
    res = pygbif.occurrences.download([f"taxonKey = {taxon_id}","mediatype = StillImage","basisOfRecord = HUMAN_OBSERVATION"],\
        user='michaltorma',\
        email='miso.torma@gmail.com',\
        pwd='etAQnyTDVUGis67dUUKKQK7PJ5oHUkYYsYF57d2',\
        )
    job_id = res[0]
    download = None
    download_path = f'data/{taxon_id}'
    os.mkdir(download_path)
    while download == None:
        try:
            download = pygbif.occurrences.download_get(job_id, path=download_path)
        except:
            pass

In [10]:
glob.glob('data/10718182/*.zip')

['data/10718182/0000665-210914110416597.zip']

In [11]:
os.mkdir('tst')

FileExistsError: [Errno 17] File exists: 'tst'

In [37]:
pygbif.occurrences.download_get('0016186-210819072339941', path='./tmp')

INFO:Download file size: 132864 bytes
INFO:On disk at ./tmp/0016186-210819072339941.zip


{'key': '0016186-210819072339941',
 'path': './tmp/0016186-210819072339941.zip',
 'size': 132864}

In [36]:
!mkdir ./tmp

In [82]:
res = pygbif.name_lookup('Abietinella abietina')
RenderJSON(res)

In [94]:
res = pygbif.search(104789)
res

{'count': 0,
 'endOfRecords': True,
 'facets': [],
 'limit': 300,
 'offset': 0,
 'results': []}

In [78]:
pygbif.occurrences.search(taxonKey=104789)

{'count': 0,
 'endOfRecords': True,
 'facets': [],
 'limit': 300,
 'offset': 0,
 'results': []}

In [87]:
import pprint

In [93]:
pprint.pprint(res, depth=3)

{'count': 0,
 'endOfRecords': True,
 'facets': [],
 'limit': 300,
 'offset': 0,
 'results': []}
